#HI

In [3]:
% reset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
# objective function
def f(x):
    f = x[0]**2 + (x[1]-3)**2
    return f
# dimension of objective function
_dim = 2

# linear constrains
def h(x):
    return 0
# number of linear constrains
_hnum = 0

# nonlinear constrains
def g1(x):
    g1 = x[1]**2 - 2*x[0]
    return g1
def g2(x):
    g2 = (x[1]-1)**2 + 5*x[0] - 15
    return g2
# number of nonlinear constrains
_gnum = 2

print('Initialize problem setting done.')

Initialize problem setting done.


In [26]:
# step size of finite difference in all directions
step = 1e-4
dx = np.ones(_dim)*step

def first_order_diff(func,x):
    "first order finit difference for gradient calculation"
    df = np.zeros(_dim)
    f0 = func(x)
    for i in range(_dim):
        dxi = np.zeros(_dim)
        dxi[i] = dx[i]
        df[i] = (func(x+dxi) - f0) / dx[i]
    return df

def first_order_diff_para(func,x,para):
    "first order finit difference for gradient calculation, with parameter"
    df = np.zeros(_dim)
    f0 = func(x,para)
    for i in range(_dim):
        dxi = np.zeros(_dim)
        dxi[i] = dx[i]
#         print( f0)
#         print(dx[i])
#         print(func(x+dxi,para))
#         print((func(x+dxi,para) - f0) / dx[i])
        df[i] = (func(x+dxi,para) - f0) / dx[i]
    return df

def second_order_diff(func,x):
    "second order finit difference for laplacian calculation"
    ddf = np.empty([_dim, _dim])
    # fill in each row
    for i in range(_dim):
        dxi = np.zeros(_dim)
        dxi[i] = dx[i]
        res = (func(x+dxi) - func(x)) / dx[i]
        ddf[i] = res
    return ddf

def Df(x):
    "value of df"
    return first_order_diff(f,x)
def DDf(x):
    "value of d(df)"
    return second_order_diff(Df,x)


def Dg1(x):
    "value of dg1"
    return first_order_diff(g1,x)
def DDg1(x):
    "value of d(dg1)"
    return second_order_diff(Dg1,x)
def Dg2(x):
    "value of dg2"
    return first_order_diff(g2,x)
def DDg2(x):
    "value of d(dg2)"
    return second_order_diff(Dg2,x)

def g(x,g_active):
    gg = [g1(x),g2(x)]
    return [gg[g_active]]
#     return [gg[idx] for idx in g_active]
def Dg(x,idx):
    "value of dg"
    return first_order_diff_para(g,x,idx)
    
def L(x,para):
    "lagrangian"
    if _hnum==0:
        lh = 0
    else:
        _lambda = para[range(_hnum)]
        lh = np.dot(_lambda,h(x))
    _mu = para[range(_hnum,_hnum+_gnum)]
    g_tmp = [g(x,i) for i in range(_gnum)]
    gg = np.zeros(_gnum)
    for i in range(_gnum):
        gg[i]=g_tmp[i][0]
    lg = np.dot(_mu,gg)
#     print(f(x),lh,lg)
    return f(x) + lh + lg

def DL(x,para):
    "derivative of lagrangian"
    return first_order_diff_para(L,x,para)

    
print('Utilities initialization done.')

Utilities initialization done.


In [6]:
def solve_qpsub(W,A,x):
    "solve QP subproblem"
    # upper left and lower left part
    M = np.concatenate((W,A),axis=0)
    # upper right and lower right part
    N = np.zeros((num_constrain,num_constrain))
    N = np.concatenate((A.T,N),axis=0)
    # whole coefficient matrix
    M = np.concatenate((M,N),axis=1)
    print('coefficient matrix:')
    print(M)

    # RHS assemble
    fk = Df(x)
    hk = g(x,g_active)
    b = np.concatenate((fk,hk),axis=0)
    print('rhs:')
    print(b)

    # STEP2, solve QP subproblem for s and para_active
    sol = np.linalg.solve(M, b)
    s = sol[range(_dim)]
    para_active = sol[range(_dim,len(sol))]
    print('solution:')
    print(s)
    print(para_active)
    return(s,para_active)
    
print('QP subproblem module done.')

QP subproblem module done.


In [7]:
def meritf(x,weight):
    "merit function for line search"
    if itr==1:
        _lambda = _lambda_new
    else:
        _lambda = max([_lambda,0.5*(_lambda_new+_lambda)])

    # penalaty of h
    weighted_sum_h = 0
    for i in range(_hnum):
        hh = h(x)**2;
        weighted_sum_h += para[i]*hh
    # penalaty of g
    weighted_sum_g = 0
    for i in range(_hnum,_hnum+num_active_g):
        gg = max([0,g1(x)]) ** 2;
        weighted_sum_g += para[i]*gg
    # merite function
    fai = f(x) + weighted_sum_h + weighted_sum_g
    print('merit',fai)
    return fai

def line_search(func,x,para,para_m,s):
    return 0.1

    _lambda = max([_lambda,0.5*(_lambda_new+_lambda)])

    t = 0.1
    b = 0.5
    _alpha = 1
    fk = func(x)
    print('111')
    gk = first_order_diff(func,x)
    print('222')
    sk = -s
    line_itr = 0
    while True:
        fai_alpha = fk + _alpha * t * np.dot(gk.T,sk)
        print('here',gk,sk,np.dot(gk.T,sk))
        print(line_itr,_alpha,fai_alpha,fk)
        if(fai_alpha<fk):
            _alpha /= 2 
        line_itr += 1
        if (fai_alpha>=fk) or (line_itr>6):
            break;
    return _alpha
print('Line search module done.')

Line search module done.


In [23]:
def bfgs(W_k,x_k,x_kp,para):
    dx = x_kp - x_k
    _lambda = para[range(_hnum)]
    _mu = para[range(_hnum,_hnum+g_active)] # non active inequality constrains also need to be considered
    y_k = DL(x_kp,para) - DL(x_k,para)
    xwx = np.dot(np.dot(dx.T,W_k),dx)
    if (np.dot(dx.T,y_k) > 0.2*xwx):
        theta_k = 1
    else:
        t_num = 0.8 * xwx
        t_den = xwx - np.dot(dx.T,y_k)
        theta_k = t_num / t_den
    wx = np.dot(W_k,dx)
    dg = theta_k*y_k + (1-theta_k) * wx
    term1 = np.dot(dg,dg.T)/np.dot(dg.T,dx)
    term2_numerator = np.dot(wx, wx.T)
    term2_denominator = np.dot(dx.T,wx)
    term2 = term2_numerator/term2_denominator
    W_kp = W_k + term1 - term2
    return W_kp
print('BFGS module done.')

BFGS module done.


In [44]:
def get_active_set(idx):
    idx = 1-idx
    return idx

In [10]:
def check_kkt(x):
    satisfy = 0
    return satisfy


In [65]:
# inital guess for x and para
x0 = np.ones(_dim) * 1
para = np.ones(_hnum+_gnum) * 1
max_itr = 200
# pick first inequality constrain as active
g_active = 0 # starts with 0
itr = 1

while True:
    print('======================iteration=================',itr)
    if (itr == 1):
        x_k = x0
        num_active_g =  1 # only one active set each time
        num_constrain = _hnum + num_active_g
        # matrix assemble
        _mu = para[range(_hnum,_hnum+_gnum)]
        W = DDf(x_k) + _mu[0] * DDg1(x_k) + _mu[1] * DDg2(x0) # not scalable here
#         W = np.eye(_dim) # not converging
#         W = DDf(x_k) # converges to a different point
        A = np.asarray([Dg1(x_k)])
        print(W)
        print(A)
        print(x_k)
        print('Para Init done.')
        # solve QP subproblem for search direction
        (s_k,para_active) = solve_qpsub(W,A,x_k)
        print('solving QP subproblem done.')
        # line search on merit for optimum step size
        _alpha = line_search(meritf,x_k,para,0,s_k)
        # update para and x
        para[_hnum+g_active] = para_active[_hnum]
        para[np.arange(0,_hnum,1)] = para_active[np.arange(0,_hnum,1)]
        x_kp = x_k +  _alpha * s_k
    else:
        # update W
        W = bfgs(W,x_k,x_kp,para)
        print('BFGS updating done.')
        # solve QP subproblem
        (s_k,para_active) = solve_qpsub(W,A,x_kp)
        print('solving QP subproblem done.')
        # update x
        para[_hnum+g_active] = para_active[_hnum]
        para[np.arange(0,_hnum,1)] = para_active[np.arange(0,_hnum,1)]
        x_kp = x_k + _alpha * s_k
    print('>>>>>>>>>>>>>>>>>>>>>>>>>solution:',x_kp)
    # check KKT condition
    if check_kkt(x_k) or (itr>max_itr):
        break;
    # update active set
    g_set = get_active_set(g_set)
    itr += 1


('======================iteration=================', 1)
[[ 2.00000008  0.        ]
 [ 0.          5.99999981]]
[[-2.      2.0001]]
[ 1.  1.]
Para Init done.
coefficient matrix:
[[ 2.00000008  0.         -2.        ]
 [ 0.          5.99999981  2.0001    ]
 [-2.          2.0001      0.        ]]
rhs:
[ 2.0001 -3.9999 -1.    ]
solution:
[ 0.12502187 -0.37495939]
[-0.87502813]
solving QP subproblem done.
('>>>>>>>>>>>>>>>>>>>>>>>>>solution:', array([ 1.01250219,  0.96250406]))
('======================iteration=================', 2)
BFGS updating done.
coefficient matrix:
[[-1.62959974 -3.62959982 -2.        ]
 [-3.62959982  2.37039999  2.0001    ]
 [-2.          2.0001      0.        ]]
rhs:
[ 2.02510437 -4.07489188 -1.0985903 ]
solution:
[ 0.4205541  -0.12873461]
[-1.12159205]
solving QP subproblem done.
('>>>>>>>>>>>>>>>>>>>>>>>>>solution:', array([ 1.04205541,  0.98712654]))
('======================iteration=================', 3)
BFGS updating done.
coefficient matrix:
[[-23.26281567 -2